In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_LV
from utils_training import LVTrainingData
from utils_training import evaluate_LVmodel

In [4]:
from behavior_model import LVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [8]:
acc_min, acc_max, acc_interval =  -9.5, 7, 0.25
acc_std_dev = 0.07
Inputs, Outputs, Weights = LVTrainingData(df)

In [10]:
## Load the train, val, test indexes

import pickle
with open('Idx05_train_val_test.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [11]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [12]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_LV(Inputs_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_LV(Inputs_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [13]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = LVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [14]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_lv_val_05.pth'
best_train_model_path = f'.\\weights\\bc_lv_train_05.pth'
best_trainval_model_path = f'.\\weights\\bc_lv_trainval_05.pth'

In [15]:
num_epochs = 2000

In [16]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_LVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_LVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/2000, Loss_mean: 386.2,      Loss_min: 379.78, Loss_max: 389.59 
      Loss_mean_val: 388.69, Loss_min_val: 381.69,      Loss_max_val: 396.83 
      Loss_mean_trainval: 388.45
Epoch 2/2000, Loss_mean: 382.77,      Loss_min: 378.27, Loss_max: 387.16 
      Loss_mean_val: 385.43, Loss_min_val: 373.66,      Loss_max_val: 392.75 
      Loss_mean_trainval: 385.18
Epoch 3/2000, Loss_mean: 378.62,      Loss_min: 368.91, Loss_max: 384.16 
      Loss_mean_val: 383.06, Loss_min_val: 375.72,      Loss_max_val: 395.75 
      Loss_mean_trainval: 382.64
Epoch 4/2000, Loss_mean: 375.53,      Loss_min: 374.12, Loss_max: 377.78 
      Loss_mean_val: 379.49, Loss_min_val: 364.34,      Loss_max_val: 388.09 
      Loss_mean_trainval: 379.12
Epoch 5/2000, Loss_mean: 372.48,      Loss_min: 368.87, Loss_max: 376.32 
      Loss_mean_val: 376.38, Loss_min_val: 355.05,      Loss_max_val: 385.91 
      Loss_mean_trainval: 376.01
Epoch 6/2000, Loss_mean: 369.33,      Loss_min: 361.69, Loss_max: 376.77 
  

Epoch 46/2000, Loss_mean: 189.78,      Loss_min: 168.36, Loss_max: 220.32 
      Loss_mean_val: 211.3, Loss_min_val: 141.31,      Loss_max_val: 287.07 
      Loss_mean_trainval: 209.25
Epoch 47/2000, Loss_mean: 189.68,      Loss_min: 169.39, Loss_max: 226.5 
      Loss_mean_val: 210.59, Loss_min_val: 143.9,      Loss_max_val: 277.51 
      Loss_mean_trainval: 208.59
Epoch 48/2000, Loss_mean: 190.91,      Loss_min: 158.72, Loss_max: 215.16 
      Loss_mean_val: 210.75, Loss_min_val: 139.68,      Loss_max_val: 293.15 
      Loss_mean_trainval: 208.86
Epoch 49/2000, Loss_mean: 188.67,      Loss_min: 179.24, Loss_max: 197.67 
      Loss_mean_val: 210.21, Loss_min_val: 141.98,      Loss_max_val: 284.13 
      Loss_mean_trainval: 208.16
Epoch 50/2000, Loss_mean: 188.8,      Loss_min: 169.69, Loss_max: 206.45 
      Loss_mean_val: 208.47, Loss_min_val: 129.13,      Loss_max_val: 291.46 
      Loss_mean_trainval: 206.59
Epoch 51/2000, Loss_mean: 186.19,      Loss_min: 163.16, Loss_max: 211.61 

Epoch 91/2000, Loss_mean: 163.07,      Loss_min: 131.56, Loss_max: 181.31 
      Loss_mean_val: 192.58, Loss_min_val: 129.71,      Loss_max_val: 302.3 
      Loss_mean_trainval: 189.76
Epoch 92/2000, Loss_mean: 166.49,      Loss_min: 120.52, Loss_max: 213.06 
      Loss_mean_val: 190.26, Loss_min_val: 117.13,      Loss_max_val: 269.9 
      Loss_mean_trainval: 187.99
Epoch 93/2000, Loss_mean: 160.54,      Loss_min: 143.77, Loss_max: 190.97 
      Loss_mean_val: 187.96, Loss_min_val: 136.96,      Loss_max_val: 243.67 
      Loss_mean_trainval: 185.34
Epoch 94/2000, Loss_mean: 159.61,      Loss_min: 142.48, Loss_max: 175.08 
      Loss_mean_val: 189.25, Loss_min_val: 113.29,      Loss_max_val: 263.06 
      Loss_mean_trainval: 186.42
Epoch 95/2000, Loss_mean: 158.71,      Loss_min: 126.95, Loss_max: 182.26 
      Loss_mean_val: 187.04, Loss_min_val: 92.69,      Loss_max_val: 281.12 
      Loss_mean_trainval: 184.34
Epoch 96/2000, Loss_mean: 159.43,      Loss_min: 140.93, Loss_max: 179.99

Epoch 136/2000, Loss_mean: 139.87,      Loss_min: 126.97, Loss_max: 162.2 
      Loss_mean_val: 177.13, Loss_min_val: 94.74,      Loss_max_val: 231.46 
      Loss_mean_trainval: 173.58
Epoch 137/2000, Loss_mean: 137.95,      Loss_min: 117.74, Loss_max: 154.51 
      Loss_mean_val: 173.7, Loss_min_val: 109.37,      Loss_max_val: 286.06 
      Loss_mean_trainval: 170.29
Epoch 138/2000, Loss_mean: 137.69,      Loss_min: 121.12, Loss_max: 155.44 
      Loss_mean_val: 171.27, Loss_min_val: 119.33,      Loss_max_val: 241.56 
      Loss_mean_trainval: 168.06
Epoch 139/2000, Loss_mean: 136.66,      Loss_min: 91.99, Loss_max: 165.71 
      Loss_mean_val: 173.44, Loss_min_val: 115.23,      Loss_max_val: 256.34 
      Loss_mean_trainval: 169.93
Epoch 140/2000, Loss_mean: 136.37,      Loss_min: 122.34, Loss_max: 163.16 
      Loss_mean_val: 172.06, Loss_min_val: 102.22,      Loss_max_val: 233.8 
      Loss_mean_trainval: 168.65
Epoch 141/2000, Loss_mean: 136.95,      Loss_min: 115.62, Loss_max: 14

Epoch 181/2000, Loss_mean: 125.25,      Loss_min: 108.32, Loss_max: 139.62 
      Loss_mean_val: 161.65, Loss_min_val: 60.1,      Loss_max_val: 226.32 
      Loss_mean_trainval: 158.18
Epoch 182/2000, Loss_mean: 124.09,      Loss_min: 121.9, Loss_max: 128.05 
      Loss_mean_val: 164.62, Loss_min_val: 90.03,      Loss_max_val: 225.38 
      Loss_mean_trainval: 160.75
Epoch 183/2000, Loss_mean: 121.91,      Loss_min: 102.03, Loss_max: 137.46 
      Loss_mean_val: 163.6, Loss_min_val: 61.03,      Loss_max_val: 227.16 
      Loss_mean_trainval: 159.62
Epoch 184/2000, Loss_mean: 124.69,      Loss_min: 101.11, Loss_max: 140.28 
      Loss_mean_val: 162.82, Loss_min_val: 99.38,      Loss_max_val: 284.46 
      Loss_mean_trainval: 159.18
Epoch 185/2000, Loss_mean: 124.53,      Loss_min: 113.87, Loss_max: 148.67 
      Loss_mean_val: 163.23, Loss_min_val: 119.63,      Loss_max_val: 224.7 
      Loss_mean_trainval: 159.54
Epoch 186/2000, Loss_mean: 124.72,      Loss_min: 107.22, Loss_max: 151.1

...

Epoch 1801/2000, Loss_mean: 11.41,      Loss_min: 10.73, Loss_max: 11.9 
      Loss_mean_val: 123.55, Loss_min_val: 45.02,      Loss_max_val: 317.45 
      Loss_mean_trainval: 112.85
Epoch 1802/2000, Loss_mean: 11.37,      Loss_min: 9.63, Loss_max: 12.81 
      Loss_mean_val: 122.66, Loss_min_val: 53.0,      Loss_max_val: 345.73 
      Loss_mean_trainval: 112.05
Epoch 1803/2000, Loss_mean: 11.48,      Loss_min: 11.05, Loss_max: 12.06 
      Loss_mean_val: 122.6, Loss_min_val: 56.51,      Loss_max_val: 305.37 
      Loss_mean_trainval: 112.0
Epoch 1804/2000, Loss_mean: 11.43,      Loss_min: 9.1, Loss_max: 12.58 
      Loss_mean_val: 127.67, Loss_min_val: 52.48,      Loss_max_val: 362.0 
      Loss_mean_trainval: 116.58
Epoch 1805/2000, Loss_mean: 11.8,      Loss_min: 9.49, Loss_max: 15.1 
      Loss_mean_val: 121.97, Loss_min_val: 50.12,      Loss_max_val: 324.84 
      Loss_mean_trainval: 111.46
Epoch 1806/2000, Loss_mean: 11.29,      Loss_min: 9.29, Loss_max: 13.75 
      Loss_mean_va

Epoch 1846/2000, Loss_mean: 11.15,      Loss_min: 8.12, Loss_max: 13.07 
      Loss_mean_val: 124.88, Loss_min_val: 46.73,      Loss_max_val: 331.78 
      Loss_mean_trainval: 114.03
Epoch 1847/2000, Loss_mean: 11.22,      Loss_min: 8.6, Loss_max: 12.76 
      Loss_mean_val: 122.2, Loss_min_val: 60.86,      Loss_max_val: 299.84 
      Loss_mean_trainval: 111.62
Epoch 1848/2000, Loss_mean: 11.3,      Loss_min: 9.12, Loss_max: 13.45 
      Loss_mean_val: 125.64, Loss_min_val: 57.94,      Loss_max_val: 323.83 
      Loss_mean_trainval: 114.73
Epoch 1849/2000, Loss_mean: 11.21,      Loss_min: 9.42, Loss_max: 13.98 
      Loss_mean_val: 128.36, Loss_min_val: 42.54,      Loss_max_val: 344.62 
      Loss_mean_trainval: 117.18
Epoch 1850/2000, Loss_mean: 11.03,      Loss_min: 9.67, Loss_max: 12.35 
      Loss_mean_val: 124.83, Loss_min_val: 74.16,      Loss_max_val: 285.62 
      Loss_mean_trainval: 113.98
Epoch 1851/2000, Loss_mean: 11.15,      Loss_min: 8.15, Loss_max: 13.78 
      Loss_mean

Epoch 1891/2000, Loss_mean: 10.69,      Loss_min: 9.75, Loss_max: 11.45 
      Loss_mean_val: 128.59, Loss_min_val: 50.91,      Loss_max_val: 336.04 
      Loss_mean_trainval: 117.35
Epoch 1892/2000, Loss_mean: 10.62,      Loss_min: 10.28, Loss_max: 11.28 
      Loss_mean_val: 125.68, Loss_min_val: 65.85,      Loss_max_val: 335.39 
      Loss_mean_trainval: 114.71
Epoch 1893/2000, Loss_mean: 10.67,      Loss_min: 9.47, Loss_max: 11.59 
      Loss_mean_val: 128.06, Loss_min_val: 75.19,      Loss_max_val: 281.88 
      Loss_mean_trainval: 116.86
Epoch 1894/2000, Loss_mean: 10.76,      Loss_min: 8.79, Loss_max: 13.36 
      Loss_mean_val: 130.78, Loss_min_val: 60.01,      Loss_max_val: 319.83 
      Loss_mean_trainval: 119.33
Epoch 1895/2000, Loss_mean: 10.64,      Loss_min: 9.74, Loss_max: 12.0 
      Loss_mean_val: 126.11, Loss_min_val: 70.45,      Loss_max_val: 300.16 
      Loss_mean_trainval: 115.09
Epoch 1896/2000, Loss_mean: 10.67,      Loss_min: 8.4, Loss_max: 13.33 
      Loss_me

Epoch 1936/2000, Loss_mean: 10.35,      Loss_min: 9.83, Loss_max: 10.73 
      Loss_mean_val: 126.55, Loss_min_val: 57.03,      Loss_max_val: 312.11 
      Loss_mean_trainval: 115.47
Epoch 1937/2000, Loss_mean: 11.06,      Loss_min: 8.47, Loss_max: 16.3 
      Loss_mean_val: 133.17, Loss_min_val: 71.74,      Loss_max_val: 307.61 
      Loss_mean_trainval: 121.52
Epoch 1938/2000, Loss_mean: 10.83,      Loss_min: 10.56, Loss_max: 11.12 
      Loss_mean_val: 126.22, Loss_min_val: 60.85,      Loss_max_val: 351.14 
      Loss_mean_trainval: 115.22
Epoch 1939/2000, Loss_mean: 11.31,      Loss_min: 10.09, Loss_max: 12.59 
      Loss_mean_val: 129.49, Loss_min_val: 43.25,      Loss_max_val: 275.24 
      Loss_mean_trainval: 118.22
Epoch 1940/2000, Loss_mean: 11.05,      Loss_min: 10.13, Loss_max: 11.92 
      Loss_mean_val: 128.97, Loss_min_val: 74.49,      Loss_max_val: 280.48 
      Loss_mean_trainval: 117.72
Epoch 1941/2000, Loss_mean: 10.91,      Loss_min: 9.38, Loss_max: 13.57 
      Loss

Epoch 1981/2000, Loss_mean: 10.34,      Loss_min: 9.71, Loss_max: 10.87 
      Loss_mean_val: 130.04, Loss_min_val: 41.78,      Loss_max_val: 337.35 
      Loss_mean_trainval: 118.62
Epoch 1982/2000, Loss_mean: 10.32,      Loss_min: 7.45, Loss_max: 11.98 
      Loss_mean_val: 143.64, Loss_min_val: 58.73,      Loss_max_val: 746.92 
      Loss_mean_trainval: 130.92
Epoch 1983/2000, Loss_mean: 10.35,      Loss_min: 8.72, Loss_max: 12.09 
      Loss_mean_val: 127.46, Loss_min_val: 57.24,      Loss_max_val: 273.3 
      Loss_mean_trainval: 116.29
Epoch 1984/2000, Loss_mean: 10.3,      Loss_min: 9.39, Loss_max: 10.65 
      Loss_mean_val: 132.6, Loss_min_val: 73.48,      Loss_max_val: 350.53 
      Loss_mean_trainval: 120.93
Epoch 1985/2000, Loss_mean: 10.65,      Loss_min: 9.78, Loss_max: 11.78 
      Loss_mean_val: 132.73, Loss_min_val: 72.97,      Loss_max_val: 347.15 
      Loss_mean_trainval: 121.09
Epoch 1986/2000, Loss_mean: 10.89,      Loss_min: 8.68, Loss_max: 13.2 
      Loss_mean_